In [1]:
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
from random import randint
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Flatten,Activation
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score
from skimage.io import imsave

In [124]:
# Daisy = 0, Dandelion = 1, Tulip = 2, Rose = 3, Sunflower = 4
types = ['Daisy','Dandelion','Tulip','Rose','Sunflower','Muro','Arbol']
tp = [0,1,2,3,4,5,6]
DAISY_DIR='PHOTOS/Margarita'
DANDI_DIR='PHOTOS/DienteLeon'
TULIP_DIR='PHOTOS/Tulipanes'
ROSE_DIR='PHOTOS/Rosas'
SUNFLOWER_DIR='PHOTOS/Girasol'
MURO_DIR = 'PHOTOS/Muro'
ARBOL_DIR = 'PHOTOS/Arbol'
DIRS = [DAISY_DIR,DANDI_DIR,TULIP_DIR,ROSE_DIR,SUNFLOWER_DIR,MURO_DIR,ARBOL_DIR]
IMGS = []
test = []

In [125]:
for i in range(len(types)):
    for img in os.listdir(DIRS[i]):
        path = os.path.join(DIRS[i],img)
        img = cv2.imread(path,cv2.IMREAD_COLOR)
        IMGS.append(np.array(img))
        test.append(tp[i])

In [127]:
X = np.array(IMGS)
X = X/255
test = np.array(test)
x_train,x_test,y_train,y_test=train_test_split(X,test,test_size=0.25)

In [128]:
model = Sequential()
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same',activation ='relu', input_shape = (150,150,3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Conv2D(filters =96, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Conv2D(filters = 96, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(7, activation = "softmax"))

In [130]:
datagen = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False, 
        samplewise_std_normalization=False, 
        zca_whitening=False,
        rotation_range=10,
        zoom_range = 0.1,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        vertical_flip=False)

datagen.fit(X)

In [131]:
model.compile(optimizer=Adam(learning_rate=0.001),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [132]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 150, 150, 32)      2432      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 75, 75, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 75, 75, 64)        18496     
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 37, 37, 64)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 37, 37, 96)        55392     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 18, 18, 96)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 18, 18, 96)       

In [133]:
batch_size=64
epochs=50 #(amb 10-15 epochs es suficient)
x_test = np.array(x_test)
y_test = np.array(y_test)
model.fit(datagen.flow(X,test, batch_size=batch_size),
                              epochs = epochs, validation_data = (x_test,y_test),
                              verbose = 1, steps_per_epoch=x_train.shape[0] // batch_size)

Epoch 1/50
37/37 [==============================] - 125s 3s/step - loss: 1.5164 - accuracy: 0.3905 - val_loss: 0.8502 - val_accuracy: 0.6683
Epoch 2/50
37/37 [==============================] - 77s 2s/step - loss: 0.7902 - accuracy: 0.7078 - val_loss: 0.5106 - val_accuracy: 0.8078
Epoch 3/50
37/37 [==============================] - 74s 2s/step - loss: 0.4653 - accuracy: 0.8464 - val_loss: 0.3571 - val_accuracy: 0.8606
Epoch 4/50
37/37 [==============================] - 71s 2s/step - loss: 0.3127 - accuracy: 0.9079 - val_loss: 1.0859 - val_accuracy: 0.7362
Epoch 5/50
37/37 [==============================] - 72s 2s/step - loss: 0.2628 - accuracy: 0.9132 - val_loss: 0.0912 - val_accuracy: 0.9799
Epoch 6/50
37/37 [==============================] - 73s 2s/step - loss: 0.1967 - accuracy: 0.9417 - val_loss: 0.0945 - val_accuracy: 0.9786
Epoch 7/50
37/37 [==============================] - 70s 2s/step - loss: 0.1115 - accuracy: 0.9672 - val_loss: 0.1096 - val_accuracy: 0.9598
Epoch 8/50
37/37 [=

In [134]:
model.save('my_model.h5')
pred = model.predict(x_test)
accuracy_score(np.argmax(pred,axis=1),y_test)